# Import all the relevant modules

In [2]:
#%matplotlib widget
%matplotlib inline

import numpy as np
np.set_printoptions(linewidth=200) #set output length, default=75

# Plotting
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size':14})
sns.set_theme()

import ipywidgets as widgets
from ipywidgets import HBox, VBox
import functools

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# About Jupyter and the following Notebooks

(Almost) every function is documented in the [Numpy docstring convention](https://numpydoc.readthedocs.io/en/latest/format.html) and the documentation can be displayed for each object individually by calling `help(<object>)` or `<object>?`, e.g. `help(np.linspace)` or `np.linspace?`. Also, for readability and good practice, many functions and sliders are defined as modules in the `Modules/` folder.

(Almost) always the initial setup is taken from the lecture, i.e. has $n = 6, t = 0.1$, etc. as default parameters.


This Notebook contains interactive widgets to change parameters on the fly and get a feeling for the different models via hands-on experience by the user. If the widgets in the notebook are **not displayed correctly** please refer to the official [Ipywidgets Documentation](https://ipywidgets.readthedocs.io/en/latest/user_install.html) for help.

Some Notebooks feature enhanced display options leveraging TeX in the background. Although all outputs work without TeX support the quality and human readability might suffer. It is therefore recommended (if not already present) to install a working Latex distribution (e.g. [MiKTeX](https://miktex.org/), [TeX Live](https://www.tug.org/texlive/), [MacTeX](http://www.tug.org/mactex/))

# Part 1 - Introduction: Classical vs Quantum Mechanical dynamics
<!---  Define a few convenience macros for bra-ket notation. -->
$\newcommand{\ket}[1]{\left\vert{#1}\right\rangle}$
$\newcommand{\bra}[1]{\left\langle{#1}\right\vert}$
$\newcommand{\braket}[2]{\left\langle{#1}\middle|{#2}\right\rangle}$
$\newcommand{\dyad}[2]{\left|{#1}\middle\rangle\middle\langle{#2}\right|}$
$\newcommand{\mela}[3]{\left\langle #1 \vphantom{#2#3} \right| #2 \left| #3 \vphantom{#1#2} \right\rangle}$
$\newcommand\dif{\mathop{}\!\mathrm{d}}$
$\newcommand\ii{\mathrm{i}}$
$\newcommand{\coloneqq}{\mathop{:=}}$

We first want to explore the difference between classical and quantum mechanical (QM) dynamics using a simple Model: One Particle on a one dimensional $n$-chain with nearest neighbor hopping and periodic boundary conditions (see image below).


[//]:# "![](Images_For_Notebooks/1D_NN_Chain.png)"

<img src="Images_For_Notebooks/1D_NN_Chain.png" height=200 />
<figcaption> Fig: 1D NN-Chain with 8 sites and a particle (blue) on site 1</figcaption>

# Markovian evolution
<a id='markovian_evolution'></a>
In the classical sense this model's dynamics might be governed by a [Markovian Process](https://en.wikipedia.org/wiki/Markov_chain) (and describe e.g. Brownian motion) where:

* The particle is always at a definite site $j \in \{1,2, \ldots, n\}$.
* At each discrete time-step it might hop with equal probability $p_1$ to the left or right, or it stays put with $p_0 = 1 - 2p_1$.
    + In general $p_i$ determines the probability of hopping $i$ steps to the left or right and $p_0 = 1 - \sum_i 2p_i$
* We assume $0 \leq p_i \leq 1\; \forall i$ and $\sum_{i=0} p_i = 1$, for a valid probability.
* The $j$-th component of the $n$-vector $(x_1, x_2, \ldots, x_n)$ is the probability of finding the particle at site $j$.




Let us first define the `Hopping_Matrix` $H$ (which in fact will prove to be our discretized Hamiltonian in QM time evolution) whose entries $H_{ij} \in \{0, 1\}$ determine if a particle can take a direct path from site $i$ to another site $j$ and afterwards the `Transfer_Matrix` $T$ which includes all hopping probabilities for this different paths. For $T$ to be a valid transition matrix, all rows sum have to equal 1, $\sum_j T_{ij} = 1\, \forall i$.

In [3]:
from Modules.Dynamical_Systems.Module_Markov import Hopping_Matrix, Transfer_Matrix

print(f"H = ", Hopping_Matrix(), "", sep="\n")
print(f"T = ", Transfer_Matrix(), sep="\n")

H = 
[[0. 1. 0. 0. 0. 1.]
 [1. 0. 1. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0.]
 [0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 1. 0. 1.]
 [1. 0. 0. 0. 1. 0.]]

T = 
[[0.98 0.01 0.   0.   0.   0.01]
 [0.01 0.98 0.01 0.   0.   0.  ]
 [0.   0.01 0.98 0.01 0.   0.  ]
 [0.   0.   0.01 0.98 0.01 0.  ]
 [0.   0.   0.   0.01 0.98 0.01]
 [0.01 0.   0.   0.   0.01 0.98]]


We are now ready to calculate the time evolution of an initial `state` $s(t=0)$ via $$\large s(t) = T^t s(0).$$

The code below adds slider to change the number of sites $n$, the hopping probabilities $p_i$ and the number of iterations $n_\mathrm{its}$. One can also choose a different initial state via the `Dropdown` menu or add a custom one by changing `User Input`. Finally choosing a filename and pressing the `Save Current Figure` button stores the plot in the folder `Figures/` .

In [4]:
from Modules.Dynamical_Systems.Module_Markov import Calc_Markov, Plot_Markov
from Modules.General.Module_Widgets_and_Sliders import Text_Box, Iterations_Slider, p1_Slider, p2_Slider, Initial_State, n_Slider, Save_Figure_Button, out
from Modules.General.Module_Widgets_and_Sliders import states_dict, set_filename, Click_Save_Figure

In [5]:
m = widgets.interactive(Plot_Markov,
        state=Initial_State,
        n_its=Iterations_Slider,
        n=n_Slider,
        p1=p1_Slider,
        p2=p2_Slider);

output_markov = widgets.Output()
filename = set_filename("Markov.pdf")
Save_Figure_Button.on_click(functools.partial(Click_Save_Figure, widget=m, name_widget=filename, output=output_markov))

display(HBox([Save_Figure_Button, filename, output_markov]))
display(VBox([Text_Box, m, out]))

# Markov Evolution by counting paths

One can also arrive at the result obtained above by explicitly simulating not one particle, but many individual particles and calculating the different probabilities by taking an ensemble average.

In detail:

1. Assume we have $n$ positions for a particle, where $s_i, s \in \{1, 2, \ldots, n\}$ is the site a particles occupies after $i$ iterations.

2. After each time step the new position of a particle $s_{i+1}$ is given by

    $$ \large
        s_{i+1} = \begin{cases} s_{i} \mod{n} & \text{with probability } 1 - 2p_1\\
                                (s_i + 1) \mod{n} & \text{with probability } p_1 \\
                                (s_i - 1) \mod{n} & \text{with probability } p_1,
                   \end{cases}
    $$
    
    to account for periodicity.

3. If we perform this calculation for $N$ particles the probability of finding a particle at site $s$ after $i$ iterations $P_i(s)$ is given by 

    $$
        \large P_i(s) = \frac{N_i(s)}{N},
    $$
    
    where $N_i(s)$ is the number of times a particle occupied position $s$ after $i$ iterations.

In [6]:
from Modules.General.Module_Widgets_and_Sliders import seed_Slider, n_paths_Slider, initial_position_Slider
from Modules.Dynamical_Systems.Module_Markov import Calc_Markov_Path, Plot_Markov_Path

In [7]:
m_path = widgets.interactive(Plot_Markov_Path,
            initial_position=initial_position_Slider,
            n_its=Iterations_Slider,
            p1=p1_Slider,
            n_paths=n_paths_Slider,
            seed=seed_Slider,
            n=n_Slider);

output_markov_path = widgets.Output()
filename = set_filename("Markov_Path.pdf")
Save_Figure_Button.on_click(functools.partial(Click_Save_Figure, widget=m_path, name_widget=filename, output=output_markov_path))

display(HBox([Save_Figure_Button, filename, output_markov_path]))
display(VBox([m_path]))

# Quantum Mechanical Time Evolution
<a id='qm_evolution'></a>
In a quantum mechanical system we have:
* A state of the system at a given time $t$ (which is represented by a wave function $\ket{\psi(t)}$) is described by a complex $n$-vector
* The particle is in a superposition of the different sites and not localized until the measurement.
* At each time-step the wave function (WF) undergoes a unitary time evolution given by
$$
    \large \ket{\psi(t)} = U \ket{\psi(0)},
$$

where $U = \exp(-\ii t H)$ is the time evolution operator. This follows from integrating the time dependent Schrödinger equation
$$
    \large \ii \frac{\dif}{\dif t} \ket{\psi(t)} = H \ket{\psi(t)}
$$

    and setting the initial state $\ket{\psi(0)}$ at $t=0$.
* The probability of measuring the particle at time $t'$ on site $j$ is given by the overlap of the WF with the site vector, i.e. the absolute square of the their inner product. By choosing the Euclidean basis $\ket{\mathrm{e}_i}$ for our sites we simply take the amplitude, i.e. the absolute square of the $j$-th component of the WF $\ket{\psi(t')}$ to arrive at the specific probability.

In [8]:
from Modules.Dynamical_Systems.Module_QM import Calc_QM, Plot_QM_Evolution
from Modules.General.Module_Widgets_and_Sliders import t_Slider

We are now ready to take a look at the quantum mechanical evolution

In [9]:
qm = widgets.interactive(Plot_QM_Evolution,
        state=Initial_State,
        n_its=Iterations_Slider,
        n=n_Slider,
        t=t_Slider);

output_qm = widgets.Output()
filename = set_filename("QM.pdf")
Save_Figure_Button.on_click(functools.partial(Click_Save_Figure, widget=qm, name_widget=filename, output=output_qm))

display(HBox([Save_Figure_Button, filename, output_qm]))
display(VBox([Text_Box, qm]))

For completeness and better comparison a side by side view of the Markovian and Quantum Mechanical models.

In [10]:
#qm.close(), m.close() #close previous widgets instances to make execution faster
#qm.open(), m.open() #open them again, to be able to display them

display(HBox([VBox([Text_Box, m]), VBox([Text_Box, qm])]))

# Time evolution using Eigenstates of the Hamiltonian $H$

Instead of directly propagating the initial state with the Time Evolution Operator $U$, we can also use the eigen-system of the Hamiltonian to achieve time evolution, if the Hamiltonian is not explicitly time dependent. 

Starting from the time-independent Schrödinger equation
$$
    \large H \ket{\phi_n} = E_n \ket{\phi_n},
$$

with $\ket{\phi_n}$ being the Eigenvector corresponding to the $n$-th Eigenvalue $E_n$, one can construct a complete orthonormal eigen-basis of the Hamiltonian $\sum_n \dyad{\phi_n}{\phi_n} = \mathbb{1}$. This Basis has to exist, as the operator is hermitian. Inserting the latter into the time evolution of a state $\ket{\psi(0)}$

$$
    \large \ket{\psi(t)} = U \ket{\psi(0)} = \exp(-\ii t H) \ket{\psi(0)},
$$
and massaging the equation, we arrive at


$${\large 
    \begin{align}
       \ket{\psi(t)} &= \exp(-\ii t H) \mathbb{1} \ket{\psi(0)} \\
                      &= \sum_n \exp(-\ii t H) \ket{\phi_n} \underbrace{\braket{\phi_n}{\psi(0)}}_{\coloneqq c_n} \\
                      &= \sum_n c_n \exp(-\ii t H) \ket{\phi_n} \\
                      &= \sum_n c_n \exp(-\ii t E_n) \ket{\phi_n},
    \end{align}
   }% 
$$

where $c_n = \braket{\phi_n}{\psi(0)}$ are the (complex) basis coefficients. In the last step we used the Eigenvalue equation of $H$ together with the fact, that a matrix exponential is [defined](https://en.wikipedia.org/wiki/Matrix_exponential) by the Taylor series of the exponential function $$\large \mathrm{e}^X = \sum_{k=0}^\infty \frac{X^n}{n!}.$$ Note, how the time evolution operator uses a matrix exponential, whereas the eigen-decomposition only relies on the exponential of a scalar.

Finally, the probability of measuring the particle on site $j$ after $m$ steps is again given by
$$
    \large P(m, j) = \left|\braket{\mathrm{e}_j}{\psi(m)}\right|^2
$$

Let us first take a look at eigenvalues of the $n=6$-ring Hamiltonian. We can see that two of the eigenvalues are doubly degenerate, which occurs due to the symmetry of hopping either once or twice to the left or right at any given position.

In [11]:
values, _ = np.linalg.eigh(Hopping_Matrix())
print(f"Eigenvalues for H with n = 6: {values}")

Eigenvalues for H with n = 6: [-2. -1. -1.  1.  1.  2.]


In [12]:
from Modules.Dynamical_Systems.Module_QM import Calc_QM_with_Eigenstates, Plot_QM_with_Eigenstates

We check our derivation by plotting once again a quantum mechanical time evolution and expecting the exact same behavior.

In [13]:
qm_Eigenstates = widgets.interactive(Plot_QM_with_Eigenstates,
        state=Initial_State,
        n_its=Iterations_Slider,
        n=n_Slider,
        t=t_Slider);

output_qm_Eigenstates = widgets.Output()
filename = set_filename("QM_Eigenstates.pdf")
Save_Figure_Button.on_click(functools.partial(Click_Save_Figure, widget=qm_Eigenstates, name_widget=filename, output=output_qm_Eigenstates))

display(HBox([Save_Figure_Button, filename, output_qm_Eigenstates]))
display(VBox([Text_Box, qm_Eigenstates]))